## Use Case

This basic tutorial describes a simple use case to highlight how easy it is to create a workflow:

* FAANG equities (Facebook, Amazon, Apple, Netflix and Google)
* Predict next day return
* 3 Walk forward train/test splits
* Features based on lagged returns 
* All model predictions used as inputs
* 3rd split used as a stacked classifier

## Import Required Modules

In [1]:
from fml.prices.yahoo_csv import YahooCSV
from fml.pipelines.standard import Standard
from fml.splits.rolling import Rolling
from fml.targets.direction import Direction
from fml.features.returns.lagged_returns import LaggedReturns
from fml.models.pycaret_blend import PycaretBlend

## Boiler Plate

It is recommended to start with boilerplate code, and simply add the appropriate modules to build your workflow.  The following code 

```
if __name__ == "__main__":
    tickers = []
    pipeline = Standard(
        stack="",
        split=None,
        workflows=[{
            "targets": [],
            "features": [],
            "transforms": [],
            "reduce": [],
            "models": [],
            "stack": {}
        }],
        algos=[],
    )
```

## Pipeline

Add appropriate modules to boilerplate code:

* Tickers: List of equities to obtain daily price data
* Pipeline: Standard pipeline (only one supported currently)
* Stack: Enables model predictions to be used as inputs to descendant models.  
* Split: 3 Walk forward train/test splits
* Workflow Target: Next day label (1 postive, 0 negative)
* Workflow Features: Lagged returns as feature inputs
* Workflow Models: Use pycaret AutoML (defaults to Random Forest)
* Workflow Stack: Last split uses a decision tree stacking classifier
* No algoritmhic signals


In [2]:
if __name__ == "__main__":
    tickers = ["FB", "AMZN", "AAPL", "NFLX", "GOOG"]
    pipeline = Standard(
        stack="all_mesh", 
        split=Rolling(n_splits=3, gap_size=1, expanding_window=False),
        workflows=[{
            "targets": [
                Direction(), 
            ],
            "features": [
                LaggedReturns(), 
            ],
            "transforms": [], 
            "reduce": [], 
            "models": [
                PycaretBlend(), 
            ],
            "stack": {  
                "target": Direction(),
                "model": PycaretBlend(classification_models=['dt']),
            }
        }],
        algos=[], 
    )

## Fit and Predict

FML handles splitting the data, labeling each observation, generating the features for each of the 3 models and the plumbing of stacking the models:

In [3]:
# Fit
prices = YahooCSV(tickers=tickers, frequency=['daily'], start_date='2014-01-01', end_date='2020-01-01', look_back=200)
pipeline = pipeline.fit(prices)

# Predict
prices = YahooCSV(tickers=tickers, frequency=['daily'], start_date='2020-01-02', look_back=200)
predictions = pipeline.transform(prices)

2022-06-07 13:49:11.311 | INFO     | fml.pipelines.standard:fit:23 - Starting Standard workflow
2022-06-07 13:49:11.324 | INFO     | fml.prices.yahoo_csv:prices:89 - Prices: Begin import process
2022-06-07 13:49:11.325 | INFO     | fml.prices.yahoo_csv:get_data:41 - Importing price data: FB
2022-06-07 13:49:11.351 | INFO     | fml.prices.yahoo_csv:get_data:41 - Importing price data: AMZN
2022-06-07 13:49:11.374 | INFO     | fml.prices.yahoo_csv:get_data:41 - Importing price data: AAPL
2022-06-07 13:49:11.402 | INFO     | fml.prices.yahoo_csv:get_data:41 - Importing price data: NFLX
2022-06-07 13:49:11.424 | INFO     | fml.prices.yahoo_csv:get_data:41 - Importing price data: GOOG
2022-06-07 13:49:12.156 | INFO     | fml.pipelines.standard:fit:88 - Target: Direction
2022-06-07 13:49:12.166 | INFO     | fml.targets.direction:transform:24 - Forward return
2022-06-07 13:49:12.191 | INFO     | fml.utils:features_fit_transform:132 - Feature: LaggedReturns
2022-06-07 13:49:12.234 | INFO     | 

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.5494,0.5627,0.5617,0.5934,0.5771,0.0957,0.0958


2022-06-07 13:49:26.584 | INFO     | fml.features.returns.lagged_returns:transform:21 - Transform: Lagged Returns
2022-06-07 13:49:26.780 | INFO     | fml.models.pycaret_blend:predict:107 - Pycaret predict
2022-06-07 13:49:27.522 | INFO     | fml.pipelines.standard:fit:88 - Target: Direction
2022-06-07 13:49:27.528 | INFO     | fml.targets.direction:transform:24 - Forward return
2022-06-07 13:49:27.550 | INFO     | fml.utils:features_fit_transform:132 - Feature: LaggedReturns
2022-06-07 13:49:27.587 | INFO     | fml.features.returns.lagged_returns:fit:17 - Fit: Lagged Returns
2022-06-07 13:49:27.607 | INFO     | fml.features.returns.lagged_returns:transform:21 - Transform: Lagged Returns
2022-06-07 13:49:27.787 | INFO     | fml.models.pycaret_blend:fit:43 - Pycaret fit


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.5317,0.5262,0.6969,0.5560,0.6185,0.0317,0.0331


2022-06-07 13:49:36.140 | INFO     | fml.features.returns.lagged_returns:transform:21 - Transform: Lagged Returns
2022-06-07 13:49:36.329 | INFO     | fml.models.pycaret_blend:predict:107 - Pycaret predict
2022-06-07 13:49:37.108 | INFO     | fml.pipelines.standard:fit:88 - Target: Direction
2022-06-07 13:49:37.117 | INFO     | fml.targets.direction:transform:24 - Forward return
2022-06-07 13:49:37.139 | INFO     | fml.utils:features_fit_transform:132 - Feature: LaggedReturns
2022-06-07 13:49:37.177 | INFO     | fml.features.returns.lagged_returns:fit:17 - Fit: Lagged Returns
2022-06-07 13:49:37.193 | INFO     | fml.features.returns.lagged_returns:transform:21 - Transform: Lagged Returns
2022-06-07 13:49:37.372 | INFO     | fml.models.pycaret_blend:fit:43 - Pycaret fit


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.4965,0.4960,0.5078,0.5168,0.5123,-0.0081,-0.0081


2022-06-07 13:49:39.907 | INFO     | fml.features.returns.lagged_returns:transform:21 - Transform: Lagged Returns
2022-06-07 13:49:39.986 | INFO     | fml.models.pycaret_blend:predict:107 - Pycaret predict


Report saved to ./rpt.html. To upload and share your report, create a free Datapane account by running `!datapane signup`.

2022-06-07 13:49:41.061 | INFO     | fml.prices.yahoo_csv:prices:89 - Prices: Begin import process
2022-06-07 13:49:41.062 | INFO     | fml.prices.yahoo_csv:get_data:41 - Importing price data: FB
2022-06-07 13:49:41.081 | INFO     | fml.prices.yahoo_csv:get_data:41 - Importing price data: AMZN
2022-06-07 13:49:41.110 | INFO     | fml.prices.yahoo_csv:get_data:41 - Importing price data: AAPL
2022-06-07 13:49:41.150 | INFO     | fml.prices.yahoo_csv:get_data:41 - Importing price data: NFLX
2022-06-07 13:49:41.189 | INFO     | fml.prices.yahoo_csv:get_data:41 - Importing price data: GOOG
2022-06-07 13:49:41.244 | INFO     | fml.features.returns.lagged_returns:transform:21 - Transform: Lagged Returns
2022-06-07 13:49:41.439 | INFO     | fml.models.pycaret_blend:predict:107 - Pycaret predict
2022-06-07 13:49:42.340 | INFO     | fml.models.pycaret_blend:predict:107 - Pycaret predict
2022-06-07 13:49:43.183 | INFO     | fml.models.pycaret_blend:predict:107 - Pycaret predict
